In [38]:
import matplotlib.pyplot as plt
import numpy as np
from   numpy.linalg import inv
import numpy.random as npr
from   pypolyagamma import PyPolyaGamma
from scipy.stats import multivariate_normal

import matplotlib.cm as cm
from tqdm import tqdm
import polars as pl

import pandas as pd
import statsmodels.api as sm
import scipy
import scipy.optimize as opt
import seaborn as sns

from bayesian_statistics.utils_2 import *
import japanize_matplotlib

from keplergl import KeplerGl


In [39]:
obsedian_df = pd.read_csv("/home/ohta/dev/bayesian_statistics/obsedian_data/obsedian.csv")

In [41]:
import geopandas as gpd
import matplotlib.pyplot as plt

# 1次メッシュ
mesh_codes = [
    '5138', '5139', 
    '5238', '5239', '5240',
    '5338', '5339', '5340',
    '5438', '5439', '5440',
    '5538', '5539', '5540', '5541'
]

gdfs = []
for mesh_code in mesh_codes:
    path_shp = f"../data/elevation_data/G04-d-11_{mesh_code}-jgd_GML"
    gdf = gpd.read_file(path_shp)
    gdfs.append(gdf)

gdf = pd.concat(gdfs, ignore_index=True)

# カラム名の対応辞書を作成
column_mapping = {
    'G04d_001': 'mesh_code_5th',
    'G04d_002': 'average_elevation',
    'G04d_003': 'maximum_elevation',
    'G04d_004': 'minimum_elevation',
    'G04d_005': 'minimum_elevation_code',
    'G04d_006': 'maximum_slope_angle',
    'G04d_007': 'maximum_slope_direction',
    'G04d_008': 'minimum_slope_angle',
    'G04d_009': 'minimum_slope_direction',
    'G04d_010': 'average_slope_angle'
}

column_dtypes = {
    'mesh_code_5th': 'str',
    'average_elevation': 'float',
    'maximum_elevation': 'float',
    'minimum_elevation': 'float',
    'minimum_elevation_code': 'str',
    'maximum_slope_angle': 'float',
    'maximum_slope_direction': 'str',
    'minimum_slope_angle': 'float',
    'minimum_slope_direction': 'str',
    'average_slope_angle': 'float'
}

# GeoDataFrameのカラム名を変更
gdf = gdf.rename(columns=column_mapping)

# "unknown"をNaNに変換
gdf = gdf.replace('unknown', np.nan)

# カラムのデータ型を変換
gdf = gdf.astype(column_dtypes)

gdf_elevation = gdf

In [35]:
path_lake_shp = "../data/lake_data/W09-05_GML"
gdf_lake = gpd.read_file(path_lake_shp)

lake_column_name = {
    'W09_001': 'lake_name', 
    'W09_002': 'lake_code',
    'W09_003': 'max_depth',
    'W09_004': 'elevation'
}
lake_column_dtypes = {
    'lake_name': 'str',
    'lake_code': 'str',
    'max_depth': 'float',
    'elevation': 'float'
}

gdf_lake = gdf_lake.rename(columns=lake_column_name)
gdf_lake = gdf_lake.astype(lake_column_dtypes)

In [47]:
# river_node_dataディレクトリから"Stream"を含むファイルを削除
!find ../data/river_node_data -type f -name "*Stream*" -exec rm {} \;

# river_stream_dataディレクトリから"RiverNode"を含むファイルを削除
!find ../data/river_stream_data -type f -name "*RiverNode*" -exec rm {} \;

In [50]:
# 都道府県コード
pref_codes = [
    "07_07", 
    "08_08", "08_09", "08_10", "08_11", "08_12", "08_13", "08_14", 
    "07_15", "08_20", 
    "08_22"
]

river_node_column_name = {
    'W05_001': 'area_code',
    'W05_011': 'elevation',
    'W05_000': 'river_node_id'
}

river_node_column_dtypes = {
    'area_code': 'str',
    'elevation': 'float',
    'river_node_id': 'str'
}
    

# river_node_data
gdfs = []
for pref_code in pref_codes:
    path_shp = f"../data/river_node_data/W05-{pref_code}_GML"
    gdf = gpd.read_file(path_shp)
    gdfs.append(gdf)

gdf_river_node = pd.concat(gdfs, ignore_index=True)

gdf_river_node = gdf_river_node.rename(columns=river_node_column_name)
gdf_river_node = gdf_river_node.astype(river_node_column_dtypes)


In [56]:
# river_stream_data
#
#  W05_001からW05_010まで
river_stream_column_name = {
    'W05_001': 'water_system_code',  # 水系域コード
    'W05_002': 'river_code',         # 河川コード
    'W05_003': 'river_type',       # 区間種別
    'W05_004': 'river_name',         # 河川名
    'W05_005': 'source_material_type', # 原典資料種別
    'W05_006': 'flow_direction',     # 流下方向判定
    'W05_007': 'river_start_node',  # 河川始点
    'W05_008': 'river_end_node',    # 河川終点
    'W05_009': 'route_start_node',  # 流路始点
    'W05_010': 'route_end_node'     # 流路終点
}

river_stream_column_dtypes = {
    'water_system_code': 'str',
    'river_code': 'str',
    'river_type': 'int',
    'river_name': 'str',
    'source_material_type': 'str',
    'flow_direction': 'bool',
    'river_start_node': 'str',
    'river_end_node': 'str',
    'route_start_node': 'str',
    'route_end_node': 'str'
}

gdfs = []
for pref_code in pref_codes:
    path_shp = f"../data/river_stream_data/W05-{pref_code}_GML"
    gdf = gpd.read_file(path_shp)
    gdfs.append(gdf)

gdf_river_stream = pd.concat(gdfs, ignore_index=True)
gdf_river_stream = gdf_river_stream.rename(columns=river_stream_column_name)
gdf_river_stream = gdf_river_stream.astype(river_stream_column_dtypes)
